In [1]:
import os
import re
import tempfile
from monty.tempfile import ScratchDir

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from llamp.atomate2.agents import MLFFMDAgent
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY", None)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4-turbo"

In [2]:
top_llm = ChatOpenAI(
    temperature=0.2,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0.0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool(
    description=re.sub(
        r"\s+",
        " ",
        """A Python shell. Use this to execute python commands.
        Input should be a valid python command.
        
        TIPS:
        - If you want to see the output of a value, you should print it out
        with `print(...)`.
        - If you write any file to a local file system, you should ALWAYS 
        print the paths to the files as well.
        - If you plot any graph, you should show the graph, save it to local 
        and print the path to the image file.
        - Combine all your output string together and print once together in
        JSON format at the end of the code.
        """
    )
)]
tools += [
    MPThermoExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MLFFMDAgent(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
    r"\s+",
    " ",
    """You name is LLaMP and you are a helpful agent that can consult 
    materials-related data through Materials Project (MP) database, arXiv, 
    Wikipedia, and run calculations/simulations via a python REPL and MP atomate2 
    computational workflow.     
        
    If you get an error, debug your code and try again. Only use 
    the output of your code to answer the question. Ask user to clarify their 
    queries if needed. Please note that you don't have direct control over MP but 
    through multiple assistant agents to help you. You need to provide complete 
    context in the input for assistants to do their job. REFINE and CRITQUE the 
    output of the assistants if needed. You can end the conversation by saying "Bye".
    """,
    )
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

In [3]:


pattern = re.compile(r'\b' + re.escape("Bye") + r'\b[.!?]*')

query = """What are the bandgaps of the following materials: Ba(PdS2)2, FePO4, DyBi2IO4, NaFe(SO4)2, Eu2B5BrO9, Sr5As3H, TlGaS2, AgSbS2, Pr3S3N, LiLaO2. Give me one value of the most stable structure for each material."""

# with ScratchDir(rootpath=os.getcwd()):
# with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
#     os.chdir(tmpdir)
#     while True:
query = query or input("Query:")
print(f"Query: {query}")
response = agent_executor.invoke(
    {
        "input": query,
    }
)
# if re.search(pattern, response["output"]):
#     break
# query = None


Query: What are the bandgaps of the following materials: Ba(PdS2)2, FePO4, DyBi2IO4, NaFe(SO4)2, Eu2B5BrO9, Sr5As3H, TlGaS2, AgSbS2, Pr3S3N, LiLaO2. Give me one value of the most stable structure for each material.


> Entering new AgentExecutor chain...
To find the bandgaps of the specified materials, I will use the Materials Project database to retrieve the information for the most stable structure of each material. I will query the database for each material one by one.

Let's start with Ba(PdS2)2.

Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of Ba(PdS2)2?To find the bandgaps of the specified materials, I will use the Materials Project database to retrieve the information for the most stable structure of each material. I will query the database for each material one by one.

Let's start with Ba(PdS2)2.

Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of th

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'formula_pretty': 'Ba(PdS2)2', 'material_id': 'mp-28967', 'band_gap': 0.7791999999999999}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of Ba(PdS2)2 is 0.779 eV."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of Ba(PdS2)2 is 0.779 eV."
}
```

> Finished chain.
{'input': 'What is the bandgap of the most stable structure of Ba(PdS2)2?', 'output': 'The bandgap of the most stable structure of Ba(PdS2)2 is 0.779 eV.'}Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of FePO4?Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of FePO4?

> Entering new AgentExecutor chain...
Thought: To find the bandgap of the most stable structure of FePO4, I need to search for materials with the formula "FePO4" that are stable and then look for the bandgap prope

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'formula_pretty': 'FePO4', 'material_id': 'mp-753395', 'band_gap': 2.1932}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of FePO4 is 2.1932 eV."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of FePO4 is 2.1932 eV."
}
```

> Finished chain.
{'input': 'What is the bandgap of the most stable structure of FePO4?', 'output': 'The bandgap of the most stable structure of FePO4 is 2.1932 eV.'}Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of DyBi2IO4?Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of DyBi2IO4?

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "DyBi2IO4",
    "sort_fields": "-energy_above_hull",
    "fields": "material_id,band_gap,is_stable",
  

...

[{'formula_pretty': 'AgSbS2', 'material_id': 'mp-3922', 'is_stable': True, 'band_gap': 1.3716000000000004}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of AgSbS2 is 1.3716 eV."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of AgSbS2 is 1.3716 eV."
}
```

> Finished chain.
{'input': 'What is the bandgap of the most stable structure of AgSbS2?', 'output': 'The bandgap of the most stable structure of AgSbS2 is 1.3716 eV.'}Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of Pr3S3N?Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of Pr3S3N?

> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Pr3S3N",
    "sort_fields": "-energy_above_hull",
    "fields": "mat

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'formula_pretty': 'Pr3S3N', 'material_id': 'mp-558940', 'is_stable': True, 'band_gap': 1.1501000000000001}]Action:
```json
{
  "action": "Final Answer",
  "action_input": "The band gap of the most stable structure of Pr3S3N is 1.1501 eV."
}
```Action:
```json
{
  "action": "Final Answer",
  "action_input": "The band gap of the most stable structure of Pr3S3N is 1.1501 eV."
}
```

> Finished chain.
{'input': 'What is the bandgap of the most stable structure of Pr3S3N?', 'output': 'The band gap of the most stable structure of Pr3S3N is 1.1501 eV.'}Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of LiLaO2?Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of LiLaO2?

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "LiLaO2",
    "sort_fields": "-energy_above_hull",
    "fiel

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

[{'formula_pretty': 'LiLaO2', 'material_id': 'mp-1020057', 'is_stable': False, 'band_gap': 3.7336}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of LiLaO2 in the Materials Project database is 3.7336 eV. However, it is important to note that this structure is not considered stable according to the database."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of LiLaO2 in the Materials Project database is 3.7336 eV. However, it is important to note that this structure is not considered stable according to the database."
}
```

> Finished chain.
{'input': 'What is the bandgap of the most stable structure of LiLaO2?', 'output': 'The bandgap of the most stable structure of LiLaO2 in the Materials Project database is 3.7336 eV. However, it is important to note that this structure is not considered stable according to the database.'}Do I need to use a tool? No
Final Answer: Here are

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

Here are the bandgaps for the most stable structures of the materials you asked about:

- Ba(PdS2)2: 0.779 eV
- FePO4: 2.1932 eV
- DyBi2IO4: 1.367 eV
- NaFe(SO4)2: 2.094 eV
- Eu2B5BrO9: 0.8801 eV
- Sr5As3H: 0.6764 eV
- TlGaS2: 1.7885 eV
- AgSbS2: 1.3716 eV
- Pr3S3N: 1.1501 eV
- LiLaO2: 3.7336 eV (Note: This structure is not considered stable according to the Materials Project database.)


In [5]:

model = "gpt-3.5-turbo-1106"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-3.5-turbo-1106
1. Ba(PdS2)2 - Bandgap: 1.3 eV
   Most stable structure: Orthorhombic

2. FePO4 - Bandgap: 2.7 eV
   Most stable structure: Orthorhombic

3. DyBi2IO4 - Bandgap: 3.2 eV
   Most stable structure: Tetragonal

4. NaFe(SO4)2 - Bandgap: 2.8 eV
   Most stable structure: Hexagonal

5. Eu2B5BrO9 - Bandgap: 2.6 eV
   Most stable structure: Orthorhombic

6. Sr5As3H - Bandgap: 1.9 eV
   Most stable structure: Orthorhombic

7. TlGaS2 - Bandgap: 2.3 eV
   Most stable structure: Orthorhombic

8. AgSbS2 - Bandgap: 1.6 eV
   Most stable structure: Tetragonal

9. Pr3S3N - Bandgap: 1.7 eV
   Most stable structure: Hexagonal

10. LiLaO2 - Bandgap: 4.3 eV
    Most stable structure: Hexagonal


In [6]:

model = "gpt-4"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-4
I'm sorry, but as an AI model developed by OpenAI, I don't have the capacity to browse the internet in real-time or access an up-to-date database. Therefore, I cannot provide the current or exact bandgap values for the materials you mentioned. However, bandgap values are generally obtained through experimental methods or theoretical calculations. These values can be found in scientific literature or materials databases.


In [8]:

model = "gemini-pro"
llm = GoogleGenerativeAI(
    temperature=0.7,
    model=f"models/{model}",
    google_api_key=GOOGLE_API_KEY,
)


response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response)

gemini-pro

